# Put Price Evaluation for BTCC.B

## Get Prices

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

# URL of the page
url = 'https://www.m-x.ca/en/trading/data/quotes?symbol=BTCC*'

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Find the table
table = soup.find('tbody', {'class': 'text-right nowrap'})

# Initialize an empty list to store the data
options_data = []

# Check if the table is found
if table:
    # Iterate over each row in the table
    for row in table.find_all('tr'):
        try:
            # Extract data for each attribute, with error handling
            strike_price = row.find('td', class_='strike_price').text.strip()
            call_bid_price = row.find('td', class_='put bid_price').text.strip()
            call_ask_price = row.find('td', class_='put ask_price').text.strip()

            # Extract expiry date from the data-row attribute
            data_row = json.loads(row['data-row'].replace('&quot;', '"'))
            expiry_date = data_row['call']['expiry_date'] if 'call' in data_row else 'N/A'

            # Append the data to the list
            options_data.append({
                'Date': expiry_date,
                'Strike': strike_price,
                'Bid': call_bid_price,
                'Ask': call_ask_price
            })
        except (AttributeError, KeyError, json.JSONDecodeError) as e:
            print(f"Error extracting data from row: {e}")
else:
    print("Table not found.")

# Convert the list to a DataFrame
df = pd.DataFrame(options_data)

# Convert 'Expiry Date' to datetime and sort by it
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by=['Date', 'Strike'], inplace=True)

# Convert 'Strike' and 'Ask' columns to numeric (float)
df['Strike'] = pd.to_numeric(df['Strike'], errors='coerce')
df['Ask'] = pd.to_numeric(df['Ask'], errors='coerce')
df['Bid'] = pd.to_numeric(df['Bid'], errors='coerce')
df

,Date,Strike,Bid,Ask
68,2024-03-15,10.0,0.0,0.05
69,2024-03-15,10.5,0.0,0.06
70,2024-03-15,11.0,0.0,0.05
71,2024-03-15,11.5,0.0,0.06
72,2024-03-15,12.0,0.0,0.05
...,...,...,...,...
309,2026-01-16,5.0,0.8,0.93
310,2026-01-16,6.0,1.3,1.69
311,2026-01-16,7.0,1.7,2.20
312,2026-01-16,8.0,2.1,2.86


## Date of Interest

## Data & Params

In [2]:
# All options expire on January 16th 2026

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz

# Retrieve historical data
ticker = "BTCC-B.TO"
start_time = (datetime.now(pytz.timezone('US/Pacific')) - timedelta(days=365*9)).strftime('%Y-%m-%d')
end_time = (datetime.now(pytz.timezone('US/Pacific'))).strftime('%Y-%m-%d')

data = yf.download(ticker, start=start_time, end=end_time, interval="1d")[['Close']]
data['Daily_Return'] = data['Close'].pct_change()

daily_volatility = data['Daily_Return'].std()
annualized_volatility = daily_volatility * np.sqrt(252)

risk_free_rate = 0.05

current_stock_price = data['Close'].iloc[-1]  
print(current_stock_price)

[*********************100%%**********************]  1 of 1 completed

14.1899995803833



C:\Users\falty\AppData\Local\Temp\ipykernel_30020\3619140308.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Daily_Return'] = data['Close'].pct_change()


## Binomial Tree

In [3]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.stats import norm

# Assuming data['Daily_Return'] contains the daily returns
historical_volatility = np.std(np.log(1 + data['Daily_Return'].dropna())) * np.sqrt(252)  # Annualized using trading days in a year
mean_return = np.mean(data['Daily_Return'].dropna()) * 252  # Annualized

historical_skewness = skew(data['Daily_Return'].dropna())
historical_kurtosis = kurtosis(data['Daily_Return'].dropna(), fisher=False)

def binomial_tree_put_option_price(S, K, T_days, r, sigma, N=20_000, dividends=0, skew=0, kurtosis=0, drift=0):
    T = T_days / 252  # Convert days to years
    dt = T / N
    u = np.exp((drift - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt))  # Incorporating drift
    d = 1 / u
    p = (np.exp((r - dividends) * dt) - d) / (u - d)

    p += skew * 0.001 * (1 - 2 * p) + kurtosis * 0.0005 * (1 - 2 * p)
    p = min(max(p, 0), 1)

    prices = S * d**np.arange(N, -1, -1) * u**np.arange(0, N+1, 1)
    option_values = np.maximum(K - prices, 0)  # Changed for put option

    discount_factor = np.exp(-r * dt)
    for i in range(N - 1, -1, -1):
        option_values[:i+1] = (p * option_values[1:i+2] + (1 - p) * option_values[:i+1]) * discount_factor
        # For American options, compare with early exercise
        option_values[:i+1] = np.maximum(option_values[:i+1], K - prices[:i+1])  # Changed for put option

    return option_values[0]

### Look for Undervalued Puts

### Look for Overvalued Puts

## Monte-Carlo Simulation 

In [4]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pytz
from numba import jit, prange

# Calculate log returns
log_returns = np.log(data['Close'] / data['Close'].shift(1)).dropna()

##### Parameter Estimation ######
# Identify jumps
jump_threshold = 3 * np.std(log_returns)
jumps = log_returns[abs(log_returns) > jump_threshold]

# Estimate lambda (jump intensity)
lambda_ = len(jumps) / (len(log_returns) / 365)

# Estimate mu_J and sigma_J (jump size parameters)
mu_J = np.mean(jumps)
sigma_J = np.std(jumps)

# Calculate historical volatility and future price changes
volatility = log_returns.rolling(window=30).std()
shifted_volatility = volatility.shift(-1)
future_price_changes = data['Close'].pct_change().shift(-1)

# Derive reflexivity factor as the correlation
reflexivity_factor = shifted_volatility.corr(future_price_changes)

# Calculate normal distribution parameters
normal_mu = np.mean(log_returns)
normal_std = np.std(log_returns)



@jit(nopython=True, parallel=True)  # Enable JIT compilation with parallel execution
def monte_carlo_put_option_price(S, K, T_months, r, sigma, lambda_, mu_J, sigma_J, reflexivity_factor, num_paths=10000, num_steps=100, skew=0, kurtosis=0):
    """
    Monte Carlo simulation for European put option pricing with jump diffusion.

    Parameters:
    S (float): Current stock price. This is the price of the underlying asset at the simulation's start.
    K (float): Strike price of the option. This is the price at which the option can be exercised.
    T_months (int): Time to maturity in months. The time until the option's expiration date.
    r (float): Risk-free interest rate. The theoretical rate of return of an investment with zero risk.
    sigma (float): Volatility of the underlying asset. Measures the stock's price fluctuations.
    lambda_ (float): Jump intensity. The frequency of jump occurrences in the stock price per year.
    mu_J (float): Mean of the logarithm of the jump size. Average size of the jumps in the stock price.
    sigma_J (float): Standard deviation of the logarithm of the jump size. Measures the variability in the size of the jumps.
    reflexivity_factor (float): Factor that quantifies the market's reflexivity, i.e., how past returns influence future volatility.
    num_paths (int): Number of simulated paths in the Monte Carlo simulation. More paths can lead to a more accurate estimation.
    num_steps (int): Number of time steps in each simulated path. Controls the granularity of the simulation.
    skew (float): Skewness of the asset returns. Defaults to 0 for no skew. Measures the asymmetry of the return distribution.
    kurtosis (float): Kurtosis of the asset returns. Defaults to 0 for normal kurtosis. Measures the 'tailedness' of the return distribution.

    Returns:
    float: Estimated price of the European put option. This is the average discounted payoff from all the simulated paths.
    """
    T = T_months / 365  # Convert maturity from months to years
    dt = T / num_steps  # Calculate time step for simulation
    discount_factor = np.exp(-r * T)  # Calculate discount factor for present value

    payoffs = np.zeros(num_paths)  # Initialize array for option payoffs
    for i in prange(num_paths):  # Parallel loop for each path
        S_t = S  # Set initial stock price for this path
        past_return = 0  # Initialize past return for reflexivity adjustment
        for j in range(num_steps):  # Loop over each time step
            z = np.random.normal(normal_mu, normal_std)  # Generate a Normal-distributed random variable
            adjusted_sigma = sigma * (1 + reflexivity_factor * past_return) # Adjust sigma based on past return and reflexivity
            sigma = adjusted_sigma
            shock = z + skew * (z ** 2 - 1)  # Adjust random variable for skew and kurtosis
            num_jumps = np.random.poisson(lambda_ * dt)  # Determine number of jumps in this time step
            jump_sum = np.sum(np.random.lognormal(mu_J, sigma_J, num_jumps) - 1)  # Calculate total jump impact
            S_t *= np.exp((r - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * shock + jump_sum)  # Update stock price
        payoffs[i] = max(K - S_t, 0)  # Calculate option payoff for this path

    return discount_factor * np.mean(payoffs)  # Compute average discounted payoff as the option price


In [5]:
print(f"Jump Threshold: {jump_threshold}")
print(f"Number of Jumps: {len(jumps)}")
print(f"Lambda (Jump Intensity): {lambda_}")
print(f"Mu_J (Mean Jump Size): {mu_J}")
print(f"Sigma_J (Standard Deviation of Jump Size): {sigma_J}")
print(f"Reflexivity Factor: {reflexivity_factor}")
print(f"Normal Distribution Mean (Mu): {normal_mu}")
print(f"Normal Distribution Scale (STDEV): {normal_std}")

Jump Threshold: 0.11258279219148902
Number of Jumps: 9
Lambda (Jump Intensity): 4.271781534460338
Mu_J (Mean Jump Size): -0.03835795717520283
Sigma_J (Standard Deviation of Jump Size): 0.1515380143286685
Reflexivity Factor: -0.03556121593585739
Normal Distribution Mean (Mu): 0.0004434234227730197
Normal Distribution Scale (STDEV): 0.03752759739716301


### Look for Undervalued Puts

### Look for Overvalued Puts

In [8]:
%%time

for date in df['Date'].unique():
    T_days = (date - pd.Timestamp.now()).days  # Calculate days to expiration

    # Skip if T_days is zero or negative
    if T_days <= 0:
        continue

    options_on_date = df[df['Date'] == date]
    options = list(options_on_date[['Strike', 'Bid']].dropna().apply(tuple, axis=1))

    for strike, bid_price in options:
        # Ensure num_steps is at least 1
        num_steps = max(1, T_days * 24)  # Assuming 24 hours in a day for finer granularity

        option_price = monte_carlo_put_option_price(
            current_stock_price, strike, T_days, risk_free_rate, historical_volatility,
            lambda_, mu_J, sigma_J, reflexivity_factor,
            num_paths=10_000, num_steps=num_steps,
            skew=historical_skewness, kurtosis=historical_kurtosis
        )

        # Check if the bid price is higher than the calculated option price
        if bid_price > option_price:
            print(f"Overvalued Put Option for expiration {date.strftime('%Y-%m-%d')}, strike {strike}: Calculated Price: ${option_price:.5f}, Bid Price: ${bid_price}")


CPU times: total: 14min 16s
Wall time: 28 s
